In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
titanic_md = pd.read_csv('titanic_MD.csv')
titanic = pd.read_csv('titanic.csv')

In [ ]:
print(titanic.shape)
titanic

In [ ]:
class fill_nas:
    def __init__(self, df):
        from sklearn.impute import SimpleImputer
        from sklearn.linear_model import LogisticRegression
        from sklearn.linear_model import LinearRegression
        self.df = df
        self.df.replace(r'\?', np.nan, regex = True, inplace = True)
        self.filled_df = self.df.copy()
        self.cols_missing_object = []
        self.cols_missing_float = []
        for col in self.df.columns:
            if self.df[col].dtype == 'object' and self.df[col].isna().sum() > 0:
                self.cols_missing_object.append(col)
            elif self.df[col].dtype == 'float' and self.df[col].isna().sum() > 0:
                self.cols_missing_float.append(col)
    
    def missing_cases(self):
        return self.df.isna().sum()
    
    def missing_percentage(self):
        return self.df.isna().sum()/self.df.shape[0]
    
    def missing_objects(self):
        return self.cols_missing_object
    
    def missing_floats(self):
        return self.cols_missing_float
    
    def listwise_deletion(self, col = 'All'):
        if col == 'All':
            return self.filled_df.dropna()
        elif type(col) is int: 
            return self.filled_df.iloc[:,col:col+1].dropna()
        elif type(col) is str:
            return self.filled_df[[col]].dropna()
    
    def pairwise_deletion(self, col_1 = 0, col_2 = 1):
        column_1 = self.df.columns[col_1]
        column_2 = self.df.columns[col_2]
        db = self.filled_df[self.filled_df[column_1].notna() & self.filled_df[column_2].notna()]
        return db
    
    def imputation(self, col = 'Sex', method = 'mean'):
        if self.filled_df[col].dtype == 'object':
            imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
            db = self.filled_df[[col]]
            return imp.fit_transform(db)
        elif self.filled_df[col].dtype == 'float':
            imp = SimpleImputer(missing_values = np.nan, strategy = method)
            db = self.filled_df[[col]]
            return imp.fit_transform(db)
        
    def sectorized_imputation(self, col = 'Sex'):
        if col == 'Sex' or col == 4:
            row_female_missing = self.filled_df[col].isna() & self.filled_df['Name'].str.contains(pat = '(Mlle|Miss|Mrs|Countess|Mme|Lady)\.',regex = True)
            row_male_missing = self.filled_df[col].isna() & self.filled_df['Name'].str.contains(pat = '(Mr|Capt|Master|Dr|Col|Major|Sir)\.',regex = True)
            self.filled_df[col] = np.where(row_female_missing == True, 'female', self.filled_df[col])
            self.filled_df[col] = np.where(row_male_missing == True, 'male', self.filled_df[col])
            db = self.filled_df[[col]]
            return db
        elif col not in ['Sex','Survived','Pclass'] or col not in [1,2,4] and col in self.df.columns:
            if self.filled_df[col].dtype == 'object':
                dic_values = pd.DataFrame(self.filled_df.groupby(['Survived', 'Pclass'])[col].agg(lambda x:x.value_counts().index[0])).reset_index()
                dic_values.columns = ['Survived', 'Pclass', 'Col']
                df = self.filled_df.merge(dic_values, how = 'left', on = ['Survived', 'Pclass'])
                df[col+'_new'] = np.where(df[col].isna(), df['Col'], df[col])
                return df[[col+'_new']]
            elif self.filled_df[col].dtype == 'float':
                dic_values = pd.DataFrame(self.filled_df.groupby(['Survived', 'Pclass'])[col].agg(lambda x:x.median())).reset_index()
                dic_values.columns = ['Survived', 'Pclass', 'Col']
                df = self.filled_df.merge(dic_values, how = 'left', on = ['Survived', 'Pclass'])
                df[col+'_new'] = np.where(df[col].isna(), df['Col'], df[col])
                return df[[col+'_new']]
    
    def predictive_model(self, col):
        if self.filled_df[col].dtype == 'object' and col in self.cols_missing_object:
            df = self.filled_df[['Survived','Pclass',col]].copy()
            df = df.dropna()
            if col == 'Sex':
                df['Sex'] = np.where(df['Sex'] == 'male',1,0)
            elif col == 'Embarked':
                df['Embarked'] = np.where(df['Embarked'] == 'S',0,df['Embarked'])
                df['Embarked'] = np.where(df['Embarked'] == 'C',1,df['Embarked'])
                df['Embarked'] = np.where(df['Embarked'] == 'Q',2,df['Embarked'])
            lr = LogisticRegression(multi_class='multinomial', solver = 'newton-cg')
            x_train = df[['Survived','Pclass']]
            y_train = df[[col]]
            lr.fit(x_train,y_train.astype('int'))
            y_pred = lr.predict(self.filled_df[['Survived','Pclass']])
            if col == 'Sex':
                y_pred = np.where(y_pred == 0,'female','male')
            elif col == 'Embarked':
                y_pred = np.where(y_pred == 0,'S',y_pred)
                y_pred = np.where(y_pred == 1,'C',y_pred)
                y_pred = np.where(y_pred == 2,'Q',y_pred)
            return y_pred
        elif self.filled_df[col].dtype == 'float' and col in self.cols_missing_float:
            df = self.filled_df[['Survived','Pclass',col]].copy()
            df = df.dropna()
            lm = LinearRegression()
            x_train = df[['Survived','Pclass']]
            y_train = df[[col]]
            lm.fit(x_train,y_train)
            y_pred = lr.predict(self.filled_df[['Survived','Pclass']])
            return y_pred
    
    def outlier_sda(self, col, sigma = 3, option = 'cap'):
        if col in self.cols_missing_float:
            avrg = np.nanmean(self.filled_df[col])
            sdev = np.nanstd(self.filled_df[col])
            upper_lim = avrg + sdev * sigma
            lower_lim = avrg - sdev * sigma
            if option == 'cap':
                capped_res = np.where(self.filled_df[col]<lower_lim,lower_lim,np.where(self.filled_df[col]>upper_lim,upper_lim,self.filled_df[col]))
                return capped_res
            else:
                deleted_res = self.filled_df[(self.filled_df[col]>=lower_lim) & (self.filled_df[col]<=upper_lim)][[col]]
                return deleted_res
            
    def outlier_pa(self, col, percentile_width = 5, option = 'cap'):
        if col in self.cols_missing_float:
            lower_lim = np.percentile(self.filled_df[col], percentile_width)
            upper_lim = np.percentile(self.filled_df[col], 100 - percentile_width)
            if option == 'cap':
                capped_res = np.where(self.filled_df[col]<lower_lim,lower_lim,np.where(self.filled_df[col]>upper_lim,upper_lim,self.filled_df[col]))
                return capped_res
            else:
                deleted_res = self.filled_df[(self.filled_df[col]>=lower_lim) & (self.filled_df[col]<=upper_lim)][[col]]
                return deleted_res

In [ ]:
class normalize_data:
    def __init__(self, df):
        from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import MinMaxScaler
        from sklearn.preprocessing import MaxAbsScaler
        self.df = df
        self.number_cols = self.df.select_dtypes(include=['float', 'int']).columns
    
    def norm_strat(strategy):
        df_normalized = self.df.copy()
        if strategy == 'standard':
            scaler1 = StandardScaler()
            for col in self.number_cols:
                df_normalized[col+'_z'] = scaler1.fit_transform(df_normalized[[col]])
            return df_normalized
        elif strategy == 'minmax':
            scaler2 = MinMaxScaler()
            for col in self.number_cols:
                df_normalized[col+'_mm'] = scaler2.fit_transform(df_normalized[[col]])
            return df_normalized
        elif strategy == 'maxabs':
            scaler3 = MaxAbsScaler()
            for col in self.number_cols:
                df_normalized[col+'_ma'] = scaler3.fit_transform(df_normalized[[col]])
            return df_normalized

In [ ]:
db_missing = fill_nas(titanic_md)

## 1.Reporte detallado de missing data para todas las columnas.
Como se puede ver a continuación, existen datos faltantes en las columnas de Sex, Age, SibSp, Parch, Fare y Embarked. Las columnas previamente mencionadas son en su mayoria columnas numericas, ya que unicamente las columnas de Sex y Embarked poseen datos textuales. De todas las columnas mencionadas anteriormente, las columnas que mas faltantes tienen son las de Age y Sex, con el 27.86% y el 13.66% de los datos totales respectivamente. 

In [ ]:
db_missing.missing_cases()

In [ ]:
db_missing.missing_percentage()

In [ ]:
db_missing.missing_objects()

In [ ]:
db_missing.missing_floats()

## 2.Para cada columna especificar que tipo de modelo se utilizará y qué valores se le darán a todos los missing values.
'Sex' = Sectorized imputation through 'Name' column, based on title.

'Embarked' = Predictive modelling using a logistic regressor based on 'Survived' and 'Pclass', used these columns since they were the only ones with complete cases.

'Age' = Sectorized imputation through 'Survived' and 'Pclass' columns using the median for each category as a result.

'SibSp' = Regular imputation using the median as a basis.

'Parch' = Regular imputation using the median as a basis.

'Fare' = Predictive modelling using a linear regressor based on 'Survived' and 'Pclass', used these columns since they were the only ones with complete cases.

## 3.Reporte de qué filas están completas
Como se podra ver en la línea inferior, al momento de eliminar todas las filas que no tienen datos completos, la longitud del set de datos se reduce a 100 filas. Esto nos indica que, al evaluar contra el tamaño original del archivo, los datos originales tienen 83 casos incompletos.

In [ ]:
titanic_md.shape[0] - db_missing.listwise_deletion('All').shape[0]

## 4.Utilizar los siguientes métodos para cada columna que contiene missing values:

## a.Listwise deletion

In [ ]:
sns.distplot(db_missing.listwise_deletion('All')['Age'])

In [ ]:
sns.distplot(db_missing.listwise_deletion('All')['Fare'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('All')['SibSp'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('All')['Parch'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('All')['Sex'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('All')['Embarked'])

## b.Pairwise deletion

In [ ]:
sns.distplot(db_missing.listwise_deletion('Age')['Age'])

In [ ]:
sns.distplot(db_missing.listwise_deletion('Fare')['Fare'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('SibSp')['SibSp'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('Parch')['Parch'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('Sex')['Sex'])

In [ ]:
sns.countplot(db_missing.listwise_deletion('Embarked')['Embarked'])

## c.Imputación General

In [ ]:
sns.distplot(db_missing.imputation('Parch','mean'))

In [ ]:
sns.distplot(db_missing.imputation('Parch','most_frequent'))

In [ ]:
sns.distplot(db_missing.imputation('Parch','median'))

In [ ]:
sns.distplot(db_missing.imputation('SibSp','mean'))

In [ ]:
sns.distplot(db_missing.imputation('SibSp','most_frequent'))

In [ ]:
sns.distplot(db_missing.imputation('SibSp','median'))

In [ ]:
sns.distplot(db_missing.imputation('Age','mean'))

In [ ]:
sns.distplot(db_missing.imputation('Age','median'))

In [ ]:
sns.distplot(db_missing.imputation('Age','most_frequent'))

In [ ]:
sns.distplot(db_missing.imputation('Fare','mean'))

In [ ]:
sns.distplot(db_missing.imputation('Fare','median'))

In [ ]:
sns.distplot(db_missing.imputation('Fare','most_frequent'))

In [ ]:
db_missing.imputation('Sex','most_frequent')

In [ ]:
db_missing.imputation('Embarked','most_frequent')

## d. Imputación Secotrizada

In [ ]:
db_missing.sectorized_imputation('Embarked')

In [ ]:
db_missing.sectorized_imputation('Age')

In [ ]:
db_missing.sectorized_imputation('Fare')

In [ ]:
db_missing.sectorized_imputation('SibSp')

In [ ]:
db_missing.sectorized_imputation('Parch')

In [ ]:
db_missing.sectorized_imputation('Sex')

## e. Modelo de Regresion

In [ ]:
db_missing.predictive_model('Sex')

In [ ]:
db_missing.predictive_model('Fare')

In [ ]:
db_missing.predictive_model('Embarked')

In [ ]:
db_missing.predictive_model('SibSp')

In [ ]:
db_missing.predictive_model('Parch')

In [ ]:
db_missing.predictive_model('Age')

## f. Outliers:Standard Deviation Approach

In [ ]:
db_missing.outlier_sda('Age')

In [ ]:
db_missing.outlier_sda('SibSp')

In [ ]:
db_missing.outlier_sda('Fare')

In [ ]:
db_missing.outlier_sda('Parch')

## g. Outliers: Percentile Approach

In [ ]:
db_missing.outlier_pa('Fare')

In [ ]:
db_missing.outlier_pa('Age')

In [ ]:
db_missing.outlier_pa('SibSp')

In [ ]:
db_missing.outlier_pa('Parch')

## 5.Al comparar los métodos del inciso 4 contra “titanic.csv”, ¿Qué método (para cada columna) se acerca más a la realidad y por qué?
Como podemos ver, el metodo de imputación sectorizado para determinar el campo de Sexo es muy bueno, fallando unicamente en un solo caso. Ademas de esto, podemos ver que para las columnas de 'Parch', 'SibSp' y 'Embarked', el metodo de Listwise deletion se acerca mas a la realidad, seguramente por el hecho de que no se pierden tantos datos. El método de mean tambien sirve como una buena imputación para la columan de fare, ya que muchos de los valores se encuentran agrupados en rangos más bajos y la falta de 8 datos no distorsiona mucho la distribución.

## 6. Conclusiones
Podemos concluir que para este set de datos, existen metodos más exitosos que otros. Los modelos regresivos no parecen servir tan bien con una cantidad tan poca de datos, lo cual causa que sus resultados sean increiblemente distorsionados y poco efectivos, como se puede ver en los resultados de las regresiones. El metodo de eliminación pairwise permite que mantengamos mucho más de los datos, reduciendo el impacto total de las variables faltantes, al igaul que manteniendo la distribución de los datos como tal. Por último, cabe mencionar que las imputaciones sectorizadas son más efectivas al encontrar un patrón consistente en las variables a utilizar para realizarla. Esta efectividad se pudo ver en la imputación de la columna 'Sex', ya que se baso mucho en el nombre de la persona y en especifico, su titulo.

## 1.	Luego del pre-procesamiento de la data con Missing Values, normalice las columnas numéricas por los métodos: 

In [ ]:
dataframe_complete = db_missing.listwise_deletion('All')

In [ ]:
normal_df = normalize_data(dataframe_complete)

In [ ]:
normal_df.norm_strat('absmax')

In [ ]:
normal_df.norm_strat('minmax')

In [ ]:
normal_df.norm_strat('standard')

## 2.Compare los estadísticos que considere más importantes para su conclusión y compare contra la data completa de “titanic.csv” (deberán de normalizar también). 

In [ ]:
total_data = normalize_data(titanic)

In [ ]:
total_data.norm_strat('standard')

In [ ]:
normal_df.norm_strat('minmax')

In [ ]:
normal_df.norm_strat('absmax')